# Support Vector Machines (SVM) Notebook

This notebook demonstrates:
- Linear vs kernel SVM on synthetic data
- Margin visualization
- Hyperparameter tuning (C, gamma)
- Kernel comparison (linear, polynomial, RBF)

See accompanying markdown files in this repo for theory.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification, make_moons, make_circles
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay
%matplotlib inline

## 1) Linear separable-ish synthetic data

In [ ]:
X, y = make_classification(n_samples=600, n_features=2, n_redundant=0, n_informative=2,
                           n_clusters_per_class=1, class_sep=1.5, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

lin_clf = Pipeline([('scaler', StandardScaler()), ('svc', LinearSVC(C=1.0, random_state=42))])
lin_clf.fit(X_train, y_train)
y_pred = lin_clf.predict(X_test)
print('LinearSVC accuracy:', accuracy_score(y_test, y_pred))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred);
plt.show()

### Margin visualization (linear SVM)

In [ ]:
# Extract linear decision boundary and margins from LinearSVC approximation via SVC linear kernel
lin_svc = Pipeline([('scaler', StandardScaler()), ('svc', SVC(kernel='linear', C=1.0, random_state=42))])
lin_svc.fit(X_train, y_train)
svc = lin_svc.named_steps['svc']
w = svc.coef_[0]
b = svc.intercept_[0]

def plot_decision_boundary_and_margin(X, y, clf, title):
    plt.figure(figsize=(6,5))
    # grid
    x_min, x_max = X[:,0].min()-1, X[:,0].max()+1
    y_min, y_max = X[:,1].min()-1, X[:,1].max()+1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300), np.linspace(y_min, y_max, 300))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    plt.contourf(xx, yy, Z, alpha=0.2, cmap=plt.cm.coolwarm)
    # data
    plt.scatter(X[:,0], X[:,1], c=y, cmap=plt.cm.coolwarm, edgecolor='k')
    # decision boundary and margins in original space require inverse scaling
    # compute line: w.x + b = 0 and margins: = ±1
    # line in scaled space; we plot via decision function contours
    ax = plt.gca()
    # Plot decision function contours
    Zf = clf.decision_function(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    cs = ax.contour(xx, yy, Zf, colors=['k','k','k'], levels=[-1,0,1], linestyles=['--','-','--'])
    ax.clabel(cs, inline=True, fontsize=8)
    plt.title(title)
    plt.xlabel('x1'); plt.ylabel('x2'); plt.tight_layout()

plot_decision_boundary_and_margin(X_test, y_test, lin_svc, 'Linear SVM: boundary and margins')
plt.show()

## 2) Nonlinear datasets: moons and circles

In [ ]:
def train_plot_kernel_svm(make_data_fn, title, param_grid=None):
    X, y = make_data_fn(noise=0.2, random_state=42, n_samples=600) if make_data_fn!=make_circles else make_data_fn(noise=0.1, factor=0.5, random_state=42, n_samples=600)
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.3, random_state=42)
    pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])
    if param_grid is None:
        param_grid = {
            'svc__kernel': ['rbf'],
            'svc__C': [0.1, 1, 10],
            'svc__gamma': ['scale', 0.1, 1]
        }
    gs = GridSearchCV(pipe, param_grid, cv=3, n_jobs=-1)
    gs.fit(X_tr, y_tr)
    print(title, 'best params:', gs.best_params_)
    best = gs.best_estimator_
    y_pred = best.predict(X_te)
    print('Accuracy:', accuracy_score(y_te, y_pred))
    ConfusionMatrixDisplay.from_predictions(y_te, y_pred); plt.show()
    # plot decision regions
    x_min, x_max = X[:,0].min()-0.5, X[:,0].max()+0.5
    y_min, y_max = X[:,1].min()-0.5, X[:,1].max()+0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300), np.linspace(y_min, y_max, 300))
    Z = best.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    plt.figure(figsize=(6,5))
    plt.contourf(xx, yy, Z, alpha=0.2, cmap=plt.cm.coolwarm)
    plt.scatter(X[:,0], X[:,1], c=y, cmap=plt.cm.coolwarm, edgecolor='k', s=12)
    plt.title(title + ' decision regions')
    plt.xlabel('x1'); plt.ylabel('x2'); plt.tight_layout(); plt.show()

train_plot_kernel_svm(make_moons, 'Moons (RBF SVM)')
train_plot_kernel_svm(make_circles, 'Circles (RBF SVM)')

## 3) Kernel comparison on moons

In [ ]:
X, y = make_moons(n_samples=600, noise=0.2, random_state=42)
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.3, random_state=42)
kernels = [
    ('linear', {'svc__kernel':['linear'], 'svc__C':[0.1,1,10]}),
    ('poly',   {'svc__kernel':['poly'], 'svc__degree':[2,3], 'svc__C':[0.1,1,10], 'svc__gamma':['scale', 0.1], 'svc__coef0':[0,1]}),
    ('rbf',    {'svc__kernel':['rbf'], 'svc__C':[0.1,1,10], 'svc__gamma':['scale', 0.1, 1]})
]
results = {}
for name, grid in kernels:
    pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])
    gs = GridSearchCV(pipe, grid, cv=3, n_jobs=-1)
    gs.fit(X_tr, y_tr)
    best = gs.best_estimator_
    acc = best.score(X_te, y_te)
    results[name] = (acc, gs.best_params_)

for k, (acc, params) in results.items():
    print(f'{k}: acc={acc:.3f}, params={params}')

## 4) Notes
- Scale inputs for kernel SVMs
- Use cross-validation to tune C/gamma/degree
- For very large datasets, consider LinearSVC or approximate kernels
- Inspect number of support vectors: best_estimator_.named_steps['svc'].n_support_